
## Задача №9

Видавець звернувся до відділу маркетингу, щоб з'ясувати передбачуваний попит на книгу. Дослідження відділу маркетингу показали:
Попит на книгу в найближчі три роки, кількість екз.	2000	3000	4000	5000
Ймовірність	0,1	0,5	0,2	0,2
Прибуток від продажу становить 9 ф. ст. за книгу. Якщо книга не продається, збитки становлять 4 ф. ст. за штуку. Якщо видавець не задовольняє попит, збитки за незадоволений попит складуть 1 ф. ст. (Для підтримки репутації фірми і майбутнього попиту).
Визначте, скільки має бути видано книг в розрахунку на трирічний період.


### Вводимо початкові дані

In [32]:
demand = [2000, 3000, 4000, 5000]
probability_demand = [0.1, 0.5, 0.2, 0.2]
profit = 9
losses_non_sale = 4
losses_not_satisfy = 1

In [33]:
import pandas as pd
import numpy as np

### Побудова дерева прийняття рішень

In [34]:
def calcDecisionTree(_tree_results, _tree_income_results):
    for i in range(len(demand)):
        temp = []
        fullExpectedIncome = 0
        fullExpectedExtIncome = 0
        for j in range(len(demand)):
            soldPerDay = min(demand[i], demand[j])
            notSoldPerDay = max(0, demand[i] - demand[j])
            expectedIncome = profit * soldPerDay
            expectedIncomeExt = expectedIncome - losses_non_sale * notSoldPerDay - losses_not_satisfy * max(0, demand[j] - demand[i])
            fullExpectedIncome += expectedIncome * probability_demand[j]
            fullExpectedExtIncome += expectedIncomeExt * probability_demand[j]
            temp.append([
                probability_demand[j],
                demand[i],
                demand[j],
                soldPerDay,
                notSoldPerDay,
                max(0, demand[j] - demand[i]),
                expectedIncome,
                expectedIncomeExt,
            ])
        _tree_results.append(temp)
        _tree_income_results.append([fullExpectedIncome, fullExpectedExtIncome])


### обчислення дерева прийняття рішень

In [35]:
tree_results = []
tree_income_results = []

calcDecisionTree(tree_results, tree_income_results)


In [36]:
for i in range(len(tree_results)):
	print(pd.DataFrame(
        tree_results[i],
        columns=[
            "Ймовірність попиту",
            "Закупка",
            "Запит",
            "Продан",
            "Не продано",
            "Незадоволений попит",
            "Очікуваний чистий дохід",
            "дохід з урахуванням втрат",
		]
	))
    

   Ймовірність попиту  Закупка  Запит  Продан  Не продано  \
0                 0.1     2000   2000    2000           0   
1                 0.5     2000   3000    2000           0   
2                 0.2     2000   4000    2000           0   
3                 0.2     2000   5000    2000           0   

   Незадоволений попит  Очікуваний чистий дохід  дохід з урахуванням втрат  
0                    0                    18000                      18000  
1                 1000                    18000                      17000  
2                 2000                    18000                      16000  
3                 3000                    18000                      15000  
   Ймовірність попиту  Закупка  Запит  Продан  Не продано  \
0                 0.1     3000   2000    2000        1000   
1                 0.5     3000   3000    3000           0   
2                 0.2     3000   4000    3000           0   
3                 0.2     3000   5000    3000           0   

   

In [37]:
print(pd.DataFrame(
    tree_income_results,
    columns=[
            "чистий дохід",
            "дохід з урахуванням витрат",
		]
))

   чистий дохід  дохід з урахуванням витрат
0       18000.0                     16500.0
1       26100.0                     25100.0
2       29700.0                     26700.0
3       31500.0                     25500.0


### Визначення оптимального рішення

In [38]:
maxExpectedIncome = np.max(tree_income_results, axis=0)[0]
alternative = np.where(tree_income_results == maxExpectedIncome)[0][0]

print(
	"Очікуваний чистий дохід максимальний: "
	"альтернатива " + str(alternative + 1) +
	". Дохід: " + str(maxExpectedIncome) +" ф. ст."
)

maxExpectedIncome = np.max(tree_income_results, axis=0)[1]
alternative = np.where(tree_income_results == maxExpectedIncome)[0][0]

print(
	"Максимальний дохід з урахуванням втрат: "
	"альтернатива " + str(alternative + 1) +
	". Дохід: " + str(maxExpectedIncome) +" ф. ст."
)

Очікуваний чистий дохід максимальний: альтернатива 4. Дохід: 31500.0 ф. ст.
Максимальний дохід з урахуванням втрат: альтернатива 3. Дохід: 26700.0 ф. ст.


### Аналіз чутливості прийняття рішень

**Перевіримо чи залежить вибір від значень ймовірностей**

In [39]:
probability_demand = [0.1, 0.3, 0.1, 0.5]
test1 = []
calcDecisionTree([], test1)

probability_demand = [0.4, 0.2, 0.2, 0.2]
test2 = []
calcDecisionTree([], test2)

probability_demand = [0.2, 0.4, 0.1, 0.3]
test3 = []
calcDecisionTree([], test3)

print(test1)
print(test2)
print(test3)

[[18000.0, 16000.0], [26100.0, 24600.0], [31500.0, 29000.0], [36000.0, 32000.0]]
[[18000.0, 16800.0], [23400.0, 21200.0], [27000.0, 22800.0], [28800.0, 21600.0]]
[[18000.0, 16500.0], [25200.0, 23700.0], [28800.0, 25300.0], [31500.0, 25500.0]]


**З результатів помітно, що при зміні ймовірностей, оптимальне рішення теж змінюється.**